In [ ]:
#%pip install protobuf==3.20.1

%pip install -U sentence-transformers
%pip install umap-learn
%pip install levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=547d4a97447617591d96a8c95d6a11c34565213b4daf030dafb9aae5307f707f
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
QPATH = "Quantlet/4-qode2desc"
PATH = f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}'

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(PATH)

In [ ]:
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import numpy as np
import umap

import matplotlib.pyplot as plt
import seaborn as sns

import torch

import nltk
nltk.download('punkt')

from sentence_transformers import SentenceTransformer

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

from scipy import stats
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sys.path.append('/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/Quantlet/3-data-preprocessing')
import importlib
import preprocessing_utils
importlib.reload(preprocessing_utils)
from preprocessing_utils import *

In [ ]:
full_dataset = pd.read_csv('../../data/preprocessed/Quantlet/full_20231001.csv')

In [ ]:
print(full_dataset.shape[0])

4829


In [ ]:
model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

In [ ]:
repos_descr = full_dataset.groupby('repo')['Description'].apply(list)
repos_script = full_dataset.groupby('repo')['code_script'].apply(list)

auth_descr = full_dataset.groupby('Authors')['Description'].apply(list)
auth_script = full_dataset.groupby('Authors')['code_script'].apply(list)

In [ ]:
full_dataset['descriptions_repo'] = full_dataset['repo'].map(repos_descr)
full_dataset['script_repo']       = full_dataset['repo'].map(repos_script)

full_dataset['descriptions_auth'] = full_dataset['Authors'].map(auth_descr)
full_dataset['script_auth']       = full_dataset['Authors'].map(auth_script)

In [ ]:
def check_len(row):
  length = 0
  if isinstance(row, list):
    length = len(row)

  return length

In [ ]:
def remove_row_description(row, column_list, column):
   new_row = row.copy()
   if isinstance(new_row[column_list], list):
      new_row[column_list].remove(new_row[column])
   return new_row[column_list]

In [ ]:
def remove_row_description(row, column_list, column):
   not_removed = True
   new_row = []
   if isinstance(row[column_list], list):
      for val in row[column_list]:
         if val == row[column]:
            if not_removed:
              not_removed=False
              continue
         new_row.append(val)

   return new_row

In [ ]:
full_dataset['descriptions_repo_len_before'] = full_dataset['descriptions_repo'].apply(check_len)
full_dataset['script_repo_len_before'] = full_dataset['script_repo'].apply(check_len)

full_dataset['descriptions_auth_len_before'] = full_dataset['descriptions_auth'].apply(check_len)
full_dataset['script_auth_len_before'] = full_dataset['script_auth'].apply(check_len)

In [ ]:
full_dataset['descriptions_repo'] = full_dataset.apply(lambda x: remove_row_description(x, column_list='descriptions_repo', column='Description'), axis=1)
full_dataset['script_repo'] = full_dataset.apply(lambda x: remove_row_description(x, column_list='script_repo', column='code_script'), axis=1)

full_dataset['descriptions_auth'] = full_dataset.apply(lambda x: remove_row_description(x, column_list='descriptions_auth', column='Description'), axis=1)
full_dataset['script_auth'] = full_dataset.apply(lambda x: remove_row_description(x, column_list='script_auth', column='code_script'), axis=1)

In [ ]:
full_dataset['descriptions_repo_len_aft'] = full_dataset['descriptions_repo'].apply(check_len)
full_dataset['script_repo_len_aft'] = full_dataset['script_repo'].apply(check_len)

full_dataset['descriptions_auth_len_aft'] = full_dataset['descriptions_auth'].apply(check_len)
full_dataset['script_auth_len_aft'] = full_dataset['script_auth'].apply(check_len)

In [ ]:
full_dataset['descriptions_repo'] = full_dataset['descriptions_repo'].apply(lambda x: ' '.join(x))
full_dataset['script_repo'] = full_dataset['script_repo'].apply(lambda x: ' '.join(x))

full_dataset['descriptions_auth'] = full_dataset['descriptions_auth'].apply(lambda x: ' '.join(x))
full_dataset['script_auth'] = full_dataset['script_auth'].apply(lambda x: ' '.join(x))

In [ ]:
full_dataset['emb_descr_repo'] = full_dataset.descriptions_repo.progress_apply(lambda x: model.encode(x))
full_dataset['emb_script_repo'] = full_dataset.script_repo.progress_apply(lambda x: model.encode(x))

full_dataset['emb_descr_auth'] = full_dataset.descriptions_auth.progress_apply(lambda x: model.encode(x))
full_dataset['emb_script_auth'] = full_dataset.script_auth.progress_apply(lambda x: model.encode(x))

In [ ]:
full_dataset.to_csv('../../data/preprocessed/Quantlet/full_embed_20231002.csv')

In [ ]:
import pickle

In [ ]:
with open(f'../../data/preprocessed/Quantlet/full_embed_20231002.pkl', 'wb') as f:
        pickle.dump(full_dataset, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(f'../../data/preprocessed/Quantlet/full_embed_20231001.pkl', 'rb') as f:
        full_dataset = pickle.load(f)

In [ ]:
RS = 42
N_NEIGHB = 10
MIN_DIST = 0.25
N_DIM = 212

In [ ]:
full_dataset['emb_descr_repo']

0       [-0.118838415, 0.048298597, -0.0025480394, -0....
1       [-0.010457277, 0.038109463, -0.05886314, -0.05...
2       [-0.010457277, 0.038109463, -0.05886314, -0.05...
3       [-0.010457277, 0.038109463, -0.05886314, -0.05...
4       [0.06825945, -0.03776655, -0.033380337, -0.036...
                              ...                        
4824    [-0.07926113, 0.05869384, -0.059998795, 0.0615...
4825    [-0.07926113, 0.05869384, -0.059998795, 0.0615...
4826    [-0.07926113, 0.05869384, -0.059998795, 0.0615...
4827    [-0.07926113, 0.05869384, -0.059998795, 0.0615...
4828    [-0.07926113, 0.05869384, -0.059998795, 0.0615...
Name: emb_descr_repo, Length: 4829, dtype: object

In [ ]:
embeddings = np.stack(full_dataset['emb_descr_repo'])

In [ ]:
# Prepare data
umap_data = umap.UMAP(n_neighbors=N_NEIGHB,
                      n_components=N_DIM,
                      min_dist=MIN_DIST,
                      metric='cosine',
                      random_state=RS).fit_transform(embeddings)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


KeyboardInterrupt: ignored

In [ ]:
umap_data.shape

In [ ]:
RS = 42
from sklearn.model_selection import train_test_split

In [ ]:

# SPLIT THE DATA
labelled_qs, test_qs = train_test_split(list(full_dataset.Quantlet.unique()),
                                     test_size=0.1,
                                     random_state=RS)
train_qs, val_qs = train_test_split(labelled_qs,
                      test_size=0.1,
                      random_state=RS)


train = full_dataset[full_dataset['Quantlet'].isin(set(train_qs))].reset_index(drop=True)
val   = full_dataset[full_dataset['Quantlet'].isin(set(val_qs))].reset_index(drop=True)
test  = full_dataset[full_dataset['Quantlet'].isin(set(test_qs))].reset_index(drop=True)

full_train = pd.concat([train, test], axis=0).sample(frac=1, random_state=RS).reset_index(drop=True)

In [ ]:
# FIX NA
test.loc[test['Quantlet'].isna(), 'Quantlet'] = 'XFGexp_rtn_SRM_2d_DOENST RUN'
train['Authors'] = train['Authors'].fillna('Unknown')
val['Authors']   = val['Authors'].fillna('Unknown')
test['Authors']  = test['Authors'].fillna('Unknown')

In [ ]:
train_dataset_json = {'version' : '1.0',
                        'data' : [{'input_sequence'  : train['code_script'].iloc[i],
                                    'output_sequence' : train['Description'].iloc[i],
                                    'repo' : train['repo'].iloc[i],
                                    'Authors' : train['Authors'].iloc[i],
                                    'Authors_emb_script': train['emb_script_auth'].iloc[i],
                                    'Authors_emb_descr': train['emb_descr_auth'].iloc[i],
                                    'repo_emb_script': train['emb_script_repo'].iloc[i],
                                    'repo_emb_descr': train['emb_descr_repo'].iloc[i],
                                   } for i in range(train.shape[0])]}
val_dataset_json = {'version' : '1.0',
                    'data' : [{'input_sequence'  : val['code_script'].iloc[i],
                                'output_sequence' : val['Description'].iloc[i],
                                'repo' : val['repo'].iloc[i],
                                'Authors' : val['Authors'].iloc[i],
                                'Authors_emb_script': val['emb_script_auth'].iloc[i],
                                'Authors_emb_descr': val['emb_descr_auth'].iloc[i],
                                'repo_emb_script': val['emb_script_repo'].iloc[i],
                                'repo_emb_descr': val['emb_descr_repo'].iloc[i],}
                               for i in range(val.shape[0])]}

full_train_dataset_json = {'version' : '1.0',
                            'data' : [{'input_sequence'  : full_train['code_script'].iloc[i],
                                        'output_sequence' : full_train['Description'].iloc[i],
                                        'repo' : full_train['repo'].iloc[i],
                                        'Authors' : full_train['Authors'].iloc[i],
                                        'Authors_emb_script': full_train['emb_script_auth'].iloc[i],
                                        'Authors_emb_descr': full_train['emb_descr_auth'].iloc[i],
                                        'repo_emb_script': full_train['emb_script_repo'].iloc[i],
                                        'repo_emb_descr': full_train['emb_descr_repo'].iloc[i],}
                                       for i in range(full_train.shape[0])]}

test_dataset_json = {'version' : '1.0',
                    'data' : [{'input_sequence'  : test['code_script'].iloc[i],
                                'output_sequence' : test['Description'].iloc[i],
                                'repo' : test['repo'].iloc[i],
                                'Authors' : test['Authors'].iloc[i],
                                'Authors_emb_script': test['emb_script_auth'].iloc[i],
                                'Authors_emb_descr': test['emb_descr_auth'].iloc[i],
                                'repo_emb_script': test['emb_script_repo'].iloc[i],
                                'repo_emb_descr': test['emb_descr_repo'].iloc[i],}
                               for i in range(test.shape[0])]}

In [ ]:
with open(f'../../data/preprocessed/Quantlet/full_train_dataset_20231001.pkl', 'wb') as f:
        pickle.dump(full_train_dataset_json, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'../../data/preprocessed/Quantlet/train_dataset_20231001.pkl', 'wb') as f:
    pickle.dump(train_dataset_json, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'../../data/preprocessed/Quantlet/val_dataset_20231001.pkl', 'wb') as f:
    pickle.dump(val_dataset_json, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'../../data/preprocessed/Quantlet/test_dataset_20231001.pkl', 'wb') as f:
    pickle.dump(test_dataset_json, f, protocol=pickle.HIGHEST_PROTOCOL)